In [1]:
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import random
import time

# Load spaCy model for NLP (run `python -m spacy download en_core_web_sm` first if not installed)
nlp = spacy.load("en_core_web_sm")

# Mock job postings (simulating data from Indian job boards like Naukri, Indeed)
mock_jobs = [
    {
        "title": "Python Developer",
        "description": "Looking for a Python developer with machine learning experience and 3+ years.",
        "location": "Bangalore",
        "id": "JOB001"
    },
    {
        "title": "Front-end Developer",
        "description": "Front-end developer position requiring JavaScript, HTML, and 2 years of experience.",
        "location": "Mumbai",
        "id": "JOB002"
    },
    {
        "title": "Data Scientist",
        "description": "Data scientist role with focus on deep learning and 5+ years of experience.",
        "location": "Delhi",
        "id": "JOB003"
    }
]

# Simulated user resume
user_resume = """
Name: [Your Name]
Software engineer with 5 years of experience in Python and machine learning.
Skills: Python, Machine Learning, Java, Data Analysis
Education: B.Tech in Computer Science, IIT Delhi
Location Preference: Bangalore, Delhi
"""

# 1. Resume Parsing Module
def parse_resume(resume_text):
    """Extract skills and key details from the resume using NLP."""
    doc = nlp(resume_text)
    skills = []
    experience = None
    location = None

    # Extract skills (simple keyword matching for demo)
    skill_keywords = ["Python", "Machine Learning", "Java", "JavaScript", "HTML", "Data Analysis"]
    for token in doc:
        if token.text in skill_keywords:
            skills.append(token.text)

    # Extract experience (basic pattern matching)
    for sent in doc.sents:
        if "years of experience" in sent.text.lower():
            experience = sent.text.split("years")[0].strip().split()[-1] + " years"

    # Extract location preference
    for sent in doc.sents:
        if "Location Preference" in sent.text:
            location = sent.text.split(":")[1].strip()

    return {
        "skills": skills,
        "experience": experience,
        "location": location,
        "summary": " ".join(skills) + " " + (experience or "")
    }

# 2. Job Search and Matching Module
def match_jobs(resume_summary, job_list):
    """Match resume to job postings using TF-IDF and cosine similarity."""
    job_descriptions = [job["description"] for job in job_list]
    texts = [resume_summary] + job_descriptions

    # Vectorize text using TF-IDF
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(texts)
    resume_vector = tfidf_matrix[0]
    job_vectors = tfidf_matrix[1:]

    # Calculate similarity
    similarities = cosine_similarity(resume_vector, job_vectors)
    matched_jobs = []

    for idx, similarity in enumerate(similarities[0]):
        matched_jobs.append({
            "job": job_list[idx],
            "score": similarity
        })

    # Sort by similarity score and filter by location (if provided)
    matched_jobs = sorted(matched_jobs, key=lambda x: x["score"], reverse=True)
    return matched_jobs

# 3. Application Generation Module
def generate_application(resume_data, job):
    """Generate a simple tailored application (simulated cover letter)."""
    cover_letter = f"""
Dear Hiring Manager,

I am excited to apply for the {job['title']} position at your company (Job ID: {job['id']}).
With my {resume_data['experience']} in {', '.join(resume_data['skills'])}, I believe I am a strong fit for this role.
The job description mentions {job['description'].split('.')[0]}, and I have hands-on experience in these areas.

Thank you for considering my application. I look forward to the opportunity to discuss further.

Sincerely,
[Your Name]
    """
    return {
        "job_id": job["id"],
        "resume": resume_data["summary"],
        "cover_letter": cover_letter.strip(),
        "status": "Ready for Submission"
    }

# 4. Submission Simulation (Manual Review Placeholder)
def submit_application(application):
    """Simulate manual submission process (due to job board terms)."""
    print("\n--- Application Preview ---")
    print(f"Job ID: {application['job_id']}")
    print("Resume Summary:", application["resume"])
    print("Cover Letter:")
    print(application["cover_letter"])
    print("Status: Pre-filled form ready for manual submission")
    return {"job_id": application["job_id"], "status": "Submitted"}

# 5. Tracking and Notification Module (Simulation)
def track_applications(applications):
    """Simulate tracking and sending notifications."""
    status_updates = ["Submitted", "Under Review", "Interview Scheduled", "Rejected"]
    tracking = {}

    for app in applications:
        # Simulate random status update
        new_status = random.choice(status_updates)
        tracking[app["job_id"]] = new_status
        print(f"\n[Notification] Job ID: {app['job_id']} - Status Updated: {new_status}")
        time.sleep(1)  # Simulate delay

    return tracking

# 6. Feedback Loop (Simplified)
def update_matching_model(matched_jobs, tracking):
    """Simulate feedback loop to improve matching (basic console output)."""
    print("\n--- Feedback Loop ---")
    for job in matched_jobs:
        job_id = job["job"]["id"]
        if job_id in tracking:
            status = tracking[job_id]
            print(f"Job ID: {job_id}, Score: {job['score']:.2f}, Outcome: {status}")
            if status == "Interview Scheduled":
                print(f"Positive feedback: Increase weight for this match.")
            elif status == "Rejected":
                print(f"Negative feedback: Adjust matching criteria.")

# Main Execution Flow
def main():
    print("=== AI-Powered Job Application Assistant Prototype ===")
    
    # Step 1: Parse Resume
    print("\n1. Parsing Resume...")
    resume_data = parse_resume(user_resume)
    print("Extracted Skills:", resume_data["skills"])
    print("Experience:", resume_data["experience"])
    print("Location Preference:", resume_data["location"])
    print("Resume Summary for Matching:", resume_data["summary"])

    # Step 2: Match Jobs
    print("\n2. Matching Jobs...")
    matched_jobs = match_jobs(resume_data["summary"], mock_jobs)
    print("Top Matched Jobs:")
    for match in matched_jobs:
        print(f"Job: {match['job']['title']} (ID: {match['job']['id']}), "
              f"Score: {match['score']:.2f}, Location: {match['job']['location']}")

    # Filter by location preference (if any)
    location_preference = resume_data["location"]
    filtered_jobs = [
        match for match in matched_jobs 
        if not location_preference or match["job"]["location"] in location_preference
    ]
    if not filtered_jobs:
        filtered_jobs = matched_jobs[:1]  # Default to top match if no location match

    # Step 3: Generate Applications
    print("\n3. Generating Applications...")
    applications = []
    for match in filtered_jobs[:2]:  # Limit to top 2 for demo
        app = generate_application(resume_data, match["job"])
        applications.append(app)

    # Step 4: Simulate Submission
    print("\n4. Submitting Applications...")
    submitted_apps = []
    for app in applications:
        submitted_app = submit_application(app)
        submitted_apps.append(submitted_app)

    # Step 5: Track Applications
    print("\n5. Tracking Applications...")
    tracking = track_applications(submitted_apps)

    # Step 6: Feedback Loop
    print("\n6. Updating Matching Model...")
    update_matching_model(filtered_jobs, tracking)

    print("\n=== Process Complete ===")

if __name__ == "__main__":
    main()

=== AI-Powered Job Application Assistant Prototype ===

1. Parsing Resume...
Extracted Skills: ['Python', 'Python', 'Java']
Experience: 5 years
Location Preference: Python, Machine Learning, Java, Data Analysis
Education
Resume Summary for Matching: Python Python Java 5 years

2. Matching Jobs...
Top Matched Jobs:
Job: Python Developer (ID: JOB001), Score: 0.30, Location: Bangalore
Job: Front-end Developer (ID: JOB002), Score: 0.05, Location: Mumbai
Job: Data Scientist (ID: JOB003), Score: 0.05, Location: Delhi

3. Generating Applications...

4. Submitting Applications...

--- Application Preview ---
Job ID: JOB001
Resume Summary: Python Python Java 5 years
Cover Letter:
Dear Hiring Manager,

I am excited to apply for the Python Developer position at your company (Job ID: JOB001).
With my 5 years in Python, Python, Java, I believe I am a strong fit for this role.
The job description mentions Looking for a Python developer with machine learning experience and 3+ years, and I have hands-